In [1]:
!pip install efficientnet_pytorch torchtoolbox

     |████████████████████████████████| 58 kB 904 kB/s 
     |████████████████████████████████| 876 kB 1.7 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16035 sha256=61cf0283b698420f470f111717ae5ee0f11aba3819a0ebcc3d6aabfd3fbb05a2
  Stored in directory: /root/.cache/pip/wheels/b7/cc/0d/41d384b0071c6f46e542aded5f8571700ace4f1eb3f1591c29
  Created wheel for lmdb: filename=lmdb-1.0.0-cp37-cp37m-linux_x86_64.whl size=276756 sha256=20d8f58a421d4ecf057e1f82f8096615f840d6955cdf2b047fcfa8ad1037c9da
  Stored in directory: /root/.cache/pip/wheels/12/e2/8d/261e2b6143cc3e0e690ccc065af68e44c43c26bc903ffa5d69
Successfully built efficientnet-pytorch lmdb
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# Imports here
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import pandas as pd
import os
import random
import math
import skimage.io
#from csv_loader import load_csv

# Tiff visualisation imports and downloads
import numpy as np
import tifffile as tiff

# For re-importing python modules
import importlib
#importlib.reload(csv_loader.py)

#for quadratic score calculator
from sklearn.metrics import cohen_kappa_score


In [3]:
#use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [4]:
# Creating ability to control how many pictures go into the training sample. For debugging / training purposes
sample_size = 10616
df = pd.read_csv('../input/prostate-cancer-grade-assessment/train.csv').copy().sample(sample_size)
df.to_csv("sample.csv", sep=",", index=False)

In [5]:
class load_csv(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)# todo remove sample for debug
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
        
    
    def __getitem__(self, index):
        image_id = self.annotations.iloc[index, 0]
        img_path = os.path.join(self.root_dir, str(image_id) +".png")
        image = torch.from_numpy(skimage.io.imread(img_path)).permute(2,0,1).float()
        
        y_label = torch.tensor(int(self.annotations.iloc[index,:]['isup_grade']))
        #isup_grade = int(self.annotations.iloc[index,:]['isup_grade'])
        
        #label = np.zeros(6).astype(np.float32)
        #y_label = label[isup_grade] = 1.
        #y_label = torch.tensor(y_label)
        
        self.transform= transforms.Compose([transforms.ToPILImage(),
                                            transforms.ToTensor()])
                                            
        if self.transform:
            image = self.transform(image)
        
        return (image, y_label, image_id)

In [6]:
# Loading csv dataset into the dataset loader function load_csv. 
dataset = load_csv(csv_file='sample.csv', root_dir='../input/prostate-cancer-tiles-4x4x128px-downsampling-4x/train_128x4x4_res1/train_128x4x4_res1')

# Creating sample subsets for validation and testing datasets
sample_size = dataset.annotations.shape[0]
train_ratio = .85
valid_ratio = .05
test_ratio = 1-(train_ratio + valid_ratio)
train_size = int(train_ratio*sample_size)
valid_size = int(valid_ratio*sample_size)
test_size = sample_size - train_size - valid_size

# Defining different datasets and respective dataloaders
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=5, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=5, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=5, shuffle=False)
entire_set_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

In [7]:
# Creating model and uploading/creating needed training components
model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=6)
model._fc = model._fc = nn.Sequential(nn.Linear(model._fc.in_features, 216),
                          nn.ReLU(),
                          nn.Linear(216, 36, bias=True),
                          nn.ReLU(),
                          nn.Linear(36, 6, bias=True),
                          nn.LogSoftmax(dim=1))


if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/checkpoints/efficientnet-b4-6ed6700e.pth



Loaded pretrained weights for efficientnet-b4


In [8]:
def validate_data_function(model, test_loader, criterion):
    test_loss = 0
    accuracy = 0
    
    for ii, (inputs, labels, image_id) in enumerate(test_loader):
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        output = model.forward(inputs)
        test_loss += criterion(output,labels.long())#.item()
        
        #ps = torch.exp(output)
        #equality = (labels.argmax(dim=1) == output.argmax(dim=1))
        equality = (labels == output.argmax(dim=1))
        accuracy += equality.type(torch.FloatTensor)
        #pred = output.cpu().data.numpy().argmax()
        #qwk = cohen_kappa_score(pred, labels, weights='quadratic')
    accuracy = accuracy.mean()
    return test_loss, accuracy, image_id

In [9]:
# Training parameters and t=0 inputs
epochs = 10
print_every = 500
steps = 0
test_loss = 0

# May the training begin!
for epoch in range(epochs):
    model.train()
    running_loss = 0
        
    for ii, (inputs, labels, image_id) in enumerate(train_loader):
        steps += 1
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
       
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()

            with torch.no_grad():
                valid_loss, accuracy, image_id = validate_data_function(model, valid_loader, criterion)
            
            print(f"Epoch: {epoch+1}/{epochs}..| "
                  f"Train loss: {running_loss/print_every:.3f}..| "
                  f"Validation loss: {valid_loss/print_every:.3f}..| "                  
                  f"Validation accuracy: {accuracy:.3f}|"
                 )
            
            running_loss = 0
            model.train()
    
    path = 'base_model.pth'
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'classifier_state_dict': model._fc.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, path)
    
    
    model.cuda() # moving model to GPU for further training

Epoch: 1/10..| Train loss: 1.665..| Validation loss: 0.502..| Validation accuracy: 35.000|
Epoch: 1/10..| Train loss: 1.613..| Validation loss: 0.308..| Validation accuracy: 39.800|
Epoch: 1/10..| Train loss: 1.581..| Validation loss: 0.351..| Validation accuracy: 44.800|
Epoch: 2/10..| Train loss: 0.601..| Validation loss: 0.427..| Validation accuracy: 32.400|
Epoch: 2/10..| Train loss: 1.510..| Validation loss: 0.301..| Validation accuracy: 46.000|
Epoch: 2/10..| Train loss: 1.460..| Validation loss: 0.319..| Validation accuracy: 41.800|
Epoch: 2/10..| Train loss: 1.430..| Validation loss: 0.284..| Validation accuracy: 50.000|
Epoch: 3/10..| Train loss: 1.064..| Validation loss: 0.270..| Validation accuracy: 52.600|
Epoch: 3/10..| Train loss: 1.365..| Validation loss: 0.373..| Validation accuracy: 47.000|
Epoch: 3/10..| Train loss: 1.301..| Validation loss: 0.251..| Validation accuracy: 59.200|
Epoch: 4/10..| Train loss: 0.217..| Validation loss: 0.286..| Validation accuracy: 47.200|

In [10]:
model.eval()
    
with torch.no_grad():
    test_loss, accuracy, image_id = validate_data_function(model, test_loader, criterion)
                
print("Test Accuracy: {}%".format(accuracy)

SyntaxError: unexpected EOF while parsing (<ipython-input-10-69e197332e0d>, line 6)